In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

We're going to compute...

### Number Theory Code

The following code is needed for the function 'pointgroup_roi'.

This function will take as input the following:
* A pair $(a,p)$, that represents the equivalence class of curves mod $p$ with trace of Frobenius equal to $a$.
* An integer $n$.


It will return an array $[[x_0,y_0],[x_1,y_2]..]$,
where the $[x_i,y_i]$ are points in the fundamental domain of $\mathbb{C}/\Lambda$,
where the lattice $\Lambda$ is the ring of integers of $\mathbb{Q}(\sqrt{a^2-4p})$.
* The ring of integers appears in every equivalence class of lattices.
* If the equivalence class contains exactly one element, then that element must be the ring of integers.

So, if we only want to make a single picture for a given equivalence class, or we're making pictures of singleton equivalence classes, then the following code is enough to obtain the point group. 

In [17]:
def gcd(a,b):
    a = abs(a)
    b = abs(b)
    if min(a,b)==0:
        return max(a,b)
    while b% a !=0:
        r = b% a
        b = a
        a = r
    return a


def gcd_list(l):
    d = 0
    for a in l:
        d = gcd(a,d)
    return d

def primefac(n):
    if n == 0:
        return {0:1}
    pf = {}
    if n < 0:
        pf[-1]=1
        n = abs(n)
    if n % 2 == 0:
        pf[2] = 0
        while n % 2 == 0:
            n = n//2
            pf[2]+=1
    if n % 3 == 0:
        pf[3] = 0
        while n % 3 == 0:
            n = n//3
            pf[3]+=1
    d = 5
    e = -1
    while d*d <= n:
        if n % d == 0:
            pf[d] = 0
            while n % d == 0:
                n = n//d
                pf[d]+=1
        d+=3+e
        e*=-1
    if n > 1:
        pf[n]=1
    return pf

def pf_to_int(pf):
    n = 1
    for p in pf:
        n*= p**pf[p]
    return n


def quad_gcd(a,b):
    pf1 = primefac(a)
    pf2 = primefac(b)
    if a == 0:
        pfgcd = {p:pf2[p]//2 for p in pf2}
        return pf_to_int(pfgcd)
    pfgcd = {}
    for p in pf1:
        if p in pf2:
            pfgcd[p] = min(pf1[p],pf2[p]//2)
    return pf_to_int(pfgcd)

def discfac(d):
    pfd = primefac(d)
    rt = pf_to_int({p:pfd[p]//2 for p in pfd})
    fd = d//(rt*rt)
    if fd % 4 > 1:
        fd*=4
        rt = rt//2
    return fd, rt

## Matrix computations
def matrixgenfromd(d):
    return np.matrix([[0,d//4],[1,-(d%4)]])

def frobmatrixfromminpoly(ap):
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    one = np.matrix([[1,0],[0,1]])
    tau = matrixgenfromd(d0)
    if d0 % 4 == 0:
        return (a//2)*one+m*tau
    else:
        return ((a+m)//2) *one + m*tau

def gcdmat(mat):
    arr = np.array(mat).reshape(1,-1)[0]
    return gcd_list([int(a) for a in arr])

def matdet(m):
    m = np.array(m)
    return m[0,0]*m[1,1]-m[0,1]*m[1,0]

def mattr(m):
    m = np.array(m)
    return m[0,0]+m[1,1]

def mwexp_frommat(m):
    t = mattr(m)
    d = matdet(m)
    return abs(quad_gcd(t,d))

def matconj(m):
    return m - mattr(m)*np.matrix([[1,0],[0,1]])

def cx_to_arr(z)->np.array:
    return np.array([z.real,z.imag])

def pointgroup_roi(ap,n):
    frob = frobmatrixfromminpoly(ap)
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    frobpowerminusone= np.linalg.matrix_power(frob,n)-np.matrix([[1,0],[0,1]])
    mw2exp = mwexp_frommat(frobpowerminusone)
    mwcycmat = frobpowerminusone//mw2exp
    cycorder = matdet(mwcycmat)
    if cycorder == 1:
        one_arr = np.array([1,0])
        taumat = matrixgenfromd(d0)
        tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
        tau_cx = np.roots(tau_cp_coefs)[0]
        tau_arr = cx_to_arr(tau_cx)
        return np.array([a*one_arr+b*tau_arr
                         for a in range(mw2exp)
                         for b in range(mw2exp)])/mw2exp
    biggenorder = mw2exp*cycorder
    generator = np.array(matconj(frobpowerminusone))
    x = generator[0,0]%biggenorder
    y = generator[1,0]%biggenorder
    xygcd = gcd(x,y)
    xg,yg = (x//xygcd,y/xygcd)
    gen1intarr = np.array([xg,yg])
    gen2aintarr = np.array([cycorder,0])
    gen2bintarr = np.array([0,cycorder])
    xyints_all = {tuple((a1*gen1intarr+a2*gen2aintarr+a3*gen2bintarr)% biggenorder)
                  for a1 in range(biggenorder)
                  for a2 in range(mw2exp)
                  for a3 in range(mw2exp)}
    one_arr = np.array([1,0])
    taumat = matrixgenfromd(d0)
    tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
    tau_cx = np.roots(tau_cp_coefs)[0]
    tau_arr = cx_to_arr(tau_cx)
    points = np.array([xy[0]*one_arr+xy[1]*tau_arr for xy in xyints_all])/biggenorder
    return points

## Examples

I'm going to use the table of precomputed elliptic curve data to find pairs $(a,p)$ that have only one element in their equivalence class.

In [18]:
ellcurvedata = pd.read_pickle('../dataframes/elliptic_curve_counts.pk')

In [19]:
ellcurvedata

,TraceFrob,Prime,Discriminant,DiscriminantFac,jInvariants,degree
ap,,,,,,
"(0, 5)",0,5,-20,"(-20, 1)",[0],2
"(1, 5)",1,5,-19,"(-19, 1)",[4],1
"(2, 5)",2,5,-16,"(-4, 2)","[1, 3]",2
"(3, 5)",3,5,-11,"(-11, 1)",[2],1
"(4, 5)",4,5,-4,"(-4, 1)",[3],1
...,...,...,...,...,...,...
"(251, 16381)",251,16381,-2523,"(-3, 29)","[6190, 6252, 6149, 8256, 6959, 1809, 10431, 10...",11
"(252, 16381)",252,16381,-2020,"(-2020, 1)","[7524, 11748, 7622, 9234, 16286, 6173, 2645, 7...",8
"(253, 16381)",253,16381,-1515,"(-1515, 1)","[7110, 8942, 10621, 1981, 9728, 3817, 4713, 58...",12


The relevant discriminants are stored in the list 'goodds'.

For each of the relevant discriminants, the dictionary goodpairs stores a list of all pairs $(a,p)$ with that discriminant, with $p < 2^{14}$ and $a$ always nonnegative.

In [20]:
goodpairs = {-7:list(ellcurvedata.loc[ellcurvedata.Discriminant == -28].index)}
goodds = [-19,  -11,   -4,   -3,  -43,   -8,  -67, -163]
for d in goodds:
    goodpairs[d] = list(ellcurvedata.loc[ellcurvedata.Discriminant == d].index)
for d in goodpairs:
    goodpairs[d].sort(key= lambda x:x[1])

For each equivalence class, we pick out the pair $(a,p)$ with smallest $p>3$ that appears in the list. *I'm taking the second smallest for $p = 7$ because the smallest is $(0,7)$ and I don't want to deal with $a=0$ yet.*

In [21]:
smallest_examples = [goodpairs[d][0+int(d==-7)] for d in goodpairs]
smallest_examples_twist = [(-ap[0],ap[1]) for ap in smallest_examples]

In [22]:
smallest_examples

[(4, 11), (1, 5), (3, 5), (4, 5), (5, 7), (1, 11), (6, 11), (1, 17), (1, 41)]

We want to keep the number of points bounded above by ~15K (I'm going to use $2^{14}$ as the actual upper bound). The number of points on an elliptic curve over $\mathbb{F}_{p^n}$ is approximately $p^n$, so we will determine largest powers $n$ such that $p^n < 2^{14}$ for the primes in the smallest examples list.

In [23]:
primeexpbounds = {p[1]:[1,p[1]] for p in smallest_examples}
for p in primeexpbounds:
    while primeexpbounds[p][1]*p<2**14:
        primeexpbounds[p][0]+=1
        primeexpbounds[p][1]*=p

In [24]:
smallest_examples_pointgroups = {ap:[pointgroup_roi(ap,n) 
                                     for n in range(1,1+primeexpbounds[ap[1]][0])] 
                                     for ap in smallest_examples+smallest_examples_twist}

In [36]:
str(smallest_examples_pointgroups[(-4,5)][1][0]).replace(' ',',')

'[0.4,0.3]'

In [27]:
import os

In [43]:
for d in goodpairs:
    os.mkdir(str(d))
    
for ap in smallest_examples_pointgroups:
    a, p = ap
    d = discfac(a*a-4*p)[0]
    imagepath = str(d)+'/'+str(a)+'_'+str(p)+'/'
    os.mkdir(imagepath)
    for n, gr in enumerate(smallest_examples_pointgroups[ap]):
        file = open(imagepath+str(n+1)+'.txt','a')
        for i in range((len(gr)//3)+1):
            stri = ''
            for vec in gr[3*i:3*(i+1)]:
                stri += '['+str(vec[0])[:10]+','+str(vec[1])[:10]+'],'
            stri+='\n'
            file.write(stri)
        file.close()


In [73]:
n = 5
gr = pointgroup_roi((-5,7),n)

In [75]:
imagepath = str(-3)+'/'+str(-5)+'_'+str(7)+'/'

In [76]:
file = open(imagepath+str(n)+'.txt','a')
for i in range((len(gr)//3)+1):
    stri = ''
    for vec in gr[3*i:3*(i+1)]:
        stri += '['+str(vec[0])[:10]+','+str(vec[1])[:10]+'],'
    stri+='\n'
    file.write(stri)
file.close()


## Algebraic point groups

In [99]:
eltoforder_mod(6,19)

8

In [100]:
[pow(8,n,19) for n in range(7)]

[1, 8, 7, 18, 11, 12, 1]

In [50]:
def qr(a,p):
    r = pow(a,p//2,p)
    if r < 2:
        return r
    else:
        return -1

def eltoforder_mod(n,p):
    if (p-1)% n != 0:
        return 0
    x = 2
    while x < p:
        xe = pow(x,(p-1)//n,p)
        xens = {pow(xe,e,p) for e in range(n)}
        if len(xens)==n:
            return xe
        x+=1
    return 0

def compute_trace(fg:tuple[int],p:int)->int:
    f,g = fg
    return - sum([qr(x**3+f*x+g,p) for x in range(p)])

def get_ellcurves_in_class(ap):
    a,p = ap
    a = int(a)
    p = int(p)
    eqns = []
    ns = p-1
    d,c = discfac(a*a-4*p)
    while qr(ns,p)>=0:
        ns-=1
    for t in range(2,p):
        ft = (-3*t)%p
        gt = (2*t)%p
        tr = compute_trace((ft,gt),p)
        if tr == a:
            eqns.append((ft,gt))
        if tr == -a:
            ft = ns*ns*ft %p
            gt = ns*ns*ns*gt %p
            eqns.append((ft,gt))
    if p % 3 == 2 and a == 0:
        eqns+= [(0,1),(0,ns)]
    elif p % 3 == 1 and d == -3:
        gen6 = eltoforder_mod(6,p)
        cands = [(0,pow(gen6,e,p)) for e in range(6)]
        eqns+=[fg for fg in cands if compute_trace(fg,p)==a]
    if p % 4 == 3 and a == 0:
        eqns+=[(1,0),(ns,0)]
    elif p % 4 == 1 and d == -4:
        gen4 = eltoforder_mod(4,p)
        cands = [(pow(gen4,e,p),0) for e in range(4)]
        eqns+=[fg for fg in cands if compute_trace(fg,p)==a]
    return eqns

In [51]:
get_ellcurves_in_class((4,5))

[(2, 0)]

In [52]:
smallest_examples_twist

[(-4, 11),
 (-1, 5),
 (-3, 5),
 (-4, 5),
 (-5, 7),
 (-1, 11),
 (-6, 11),
 (-1, 17),
 (-1, 41)]

In [55]:
ecs_smex_tw = {ap:get_ellcurves_in_class(ap)[0] for ap in smallest_examples_twist}

In [56]:
ecs_smex_tw

{(-4, 11): (5, 7),
 (-1, 5): (2, 1),
 (-3, 5): (1, 1),
 (-4, 5): (3, 0),
 (-5, 7): (0, 3),
 (-1, 11): (3, 2),
 (-6, 11): (1, 3),
 (-1, 17): (8, 16),
 (-1, 41): (35, 29)}

In [60]:
def fg_to_str(fg,p=None):
    ec = 'y^2 = x^3 + '
    if fg[0]!=0:
        ec+=str(fg[0])+'x'
        if fg[1]!=0:
            ec+='+'
    if fg[1]!=0:
        ec+=str(fg[1])
    if p!= None:
        ec+=' mod '+str(p)
    return ec